## Normalization Stages (1NF, 2NF, 3NF, and BCNF)
 - Normalization is the process of organizing data in a database to reduce redundancy and dependency. 
 - This is done by dividing large tables into smaller ones and ensuring that data is stored logically. 
 - Let's go through each normal form (1NF, 2NF, 3NF, and BCNF) one by one.


### 1. First Normal Form (1NF)
A relation is in 1NF if it meets the following conditions:

- Atomicity: Each column contains atomic values (indivisible values). This means no repeating groups or arrays in a single column.

- Uniqueness: Each row in the table must be unique, and no two rows can have exactly the same data.

Example of Non-1NF:

In [ ]:
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    items VARCHAR(255)  -- This column has a list of items
);

INSERT INTO orders (order_id, customer_name, items)
VALUES (1, 'John Doe', 'Item1, Item2'),
       (2, 'Jane Smith', 'Item3, Item4');

Here, the items column is not atomic because it stores multiple values in a single field.

Converting to 1NF:
To bring the table to 1NF, we separate the repeating group (items) into individual rows.

In [ ]:
CREATE TABLE orders (
    order_id INT,
    customer_name VARCHAR(100),
    item VARCHAR(100),  -- Atomic column for items
    PRIMARY KEY (order_id, item)  -- Composite primary key
);

INSERT INTO orders (order_id, customer_name, item)
VALUES (1, 'John Doe', 'Item1'),
       (1, 'John Doe', 'Item2'),
       (2, 'Jane Smith', 'Item3'),
       (2, 'Jane Smith', 'Item4');


### 2. Second Normal Form (2NF)
A relation is in 2NF if:

- It is in 1NF.

- It has no partial dependency; that is, all non-key attributes must be fully functionally dependent on the entire primary key, not just part of it.

Example of Non-2NF:

Let's say we have the following table, where order_id and product_id together form a composite primary key.

In [ ]:
CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    product_name VARCHAR(100),
    quantity INT,
    PRIMARY KEY (order_id, product_id)
);

INSERT INTO order_items (order_id, product_id, product_name, quantity)
VALUES (1, 101, 'Laptop', 2),
       (1, 102, 'Smartphone', 3),
       (2, 103, 'Tablet', 1);


Here, product_name depends only on product_id, but the primary key is a composite of order_id and product_id. This is a partial dependency, which violates 2NF.

Converting to 2NF:

We remove the partial dependency by creating a new table for products, leaving product_name in the new table.

In [ ]:
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(100)
);

CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

INSERT INTO products (product_id, product_name)
VALUES (101, 'Laptop'),
       (102, 'Smartphone'),
       (103, 'Tablet');


### 3. Third Normal Form (3NF)
A relation is in `3NF` if:

- It is in `2NF`.

- It has no `transitive dependency`; that is, non-key attributes should depend only on the primary key, not on other non-key attributes.

Example of Non-3NF:

Let's say we have the following table, where employee_id is the primary key:

In [ ]:
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    employee_name VARCHAR(100),
    department_id INT,
    department_name VARCHAR(100),
    department_head VARCHAR(100)
);

INSERT INTO employees (employee_id, employee_name, department_id, department_name, department_head)
VALUES (1, 'John Doe', 101, 'HR', 'Alice'),
       (2, 'Jane Smith', 102, 'IT', 'Bob');

Here, department_name and department_head are dependent on department_id, but department_id is dependent on employee_id. This is a transitive dependency.

Converting to 3NF:

We remove the transitive dependency by creating a separate table for departments

In [ ]:
CREATE TABLE departments (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(100),
    department_head VARCHAR(100)
);

CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    employee_name VARCHAR(100),
    department_id INT,
    FOREIGN KEY (department_id) REFERENCES departments(department_id)
);

INSERT INTO departments (department_id, department_name, department_head)
VALUES (101, 'HR', 'Alice'),
       (102, 'IT', 'Bob');


## Constraints in PostgreSQL

Constraints help ensure data integrity, prevent invalid data, and enforce business rules at the database level.

### ✅ 1. NOT NULL Constraint

🔍 Definition
- Ensures that a column cannot contain `NULL` values.

📌 Use Case
- When a field is mandatory—like email, username, or date_of_birth in a user profile.

🧪 Example

In [ ]:
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    username VARCHAR(100) NOT NULL,
    email VARCHAR(150) NOT NULL
);

💡 Best Practices
- Use `NOT NULL` on all essential fields.

Avoid using nullable columns in primary or foreign keys.

🎯 Interview Question

Q: Can a `NOT NULL` column be part of a composite key?

A: Yes. In fact, all columns in a primary/composite key must be `NOT NULL`.

----------

### ✅ 2. UNIQUE Constraint

🔍 Definition
- Ensures that all values in a column (or a group of columns) are distinct.

📌 Use Case
- For fields like `email`, `phone_number`, `Aadhar number`, etc., where duplicates aren't allowed.

🧪 Example

In [ ]:
CREATE TABLE employees (
    emp_id SERIAL PRIMARY KEY,
    email VARCHAR(150) UNIQUE,
    phone_number VARCHAR(15) UNIQUE
);

💡 Best Practices
- Use `UNIQUE` constraints to prevent logical duplication.
- Use `UNIQUE` (col1, col2) for combined uniqueness (e.g., user_id + role).

🎯 Interview Question

 Q: Can a table have multiple UNIQUE constraints?

A: Yes, you can have multiple UNIQUE constraints on different columns.

-----------

### ✅ 3. PRIMARY KEY Constraint

🔍 Definition
- Uniquely identifies each row in the table. Combines `NOT NULL` + `UNIQUE`.

📌 Use Case
- Always required for identifying records. Use auto-incrementing `SERIAL` or `BIGSERIAL` types.

🧪 Example


In [ ]:
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(100) NOT NULL
);

💡 Best Practices
- Prefer single-column primary keys when possible.

- Composite keys should be used when a natural key combination is better than surrogate keys.

🎯 Interview Question

Q: How is a `PRIMARY KEY` different from `UNIQUE + NOT NULL`?

A: Technically same, but only one primary key is allowed per table and it clearly indicates the main identifier.

--------------

### ✅ 4. FOREIGN KEY Constraint

🔍 Definition
- Links a column (or columns) in one table to the `PRIMARY KEY` or `UNIQUE` constraint of another table.

📌 Use Case
- Maintains referential integrity. For example, orders should reference a valid customer.

🧪 Example

In [ ]:
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL
);

CREATE TABLE orders (
    order_id SERIAL PRIMARY KEY,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);


💡 Best Practices
- Index foreign key columns for performance.

- Use `ON DELETE CASCADE` or `SET NULL` to define dependent record behavior.


In [ ]:
FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    ON DELETE CASCADE


🎯 Interview Question
Q: What happens if you delete a parent row referenced by a foreign key?

A: By default, it throws an error unless you use `ON DELETE` rules like `CASCADE` or `SET NULL`.

------------

### ✅ 5. CHECK Constraint

🔍 Definition
- Allows you to enforce custom validations using expressions.

📌 Use Case
- Age must be > 18, salary must be positive, quantity must be between 1 and 100, etc.

🧪 Example


In [ ]:
CREATE TABLE employees (
    emp_id SERIAL PRIMARY KEY,
    age INT CHECK (age >= 18),
    salary NUMERIC(10, 2) CHECK (salary > 0)
);

💡 Best Practices
- Use `CHECK` for validations that are strictly rule-based.
- Avoid overcomplicating CHECK logic.

🎯 Interview Question

Q: Can you write a `CHECK` constraint to allow only specific departments?

A:

In [ ]:
CHECK (department IN ('HR', 'IT', 'Finance'))

--------------

### ✅ 6. DEFAULT Constraint

🔍 Definition
- Automatically assigns a default value to a column when no value is provided.

📌 Use Case
- Set default status = ‘active’, created_at = current date, etc.

🧪 Example

In [ ]:
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    status VARCHAR(20) DEFAULT 'active',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

💡 Best Practices

- Use `DEFAULT` for timestamps, status fields, or counters.
- Combine with `NOT NULL` to ensure the field is always populated.

🎯 Interview Question

Q: Can you override a `DEFAULT` value during insertion?

A: Yes, by explicitly providing the value.

---------------------

### ✅ 7. Composite / Multi-Column Constraints

🔍 Definition
- Constraints that involve multiple columns together, often used for `UNIQUE` or `PRIMARY KEY`.

📌 Use Case
- A student can enroll only once in a course → student_id + course_id must be unique.

🧪 Example

In [ ]:
CREATE TABLE enrollments (
    student_id INT,
    course_id INT,
    PRIMARY KEY (student_id, course_id)  -- Composite Key
);

Or:

In [ ]:
UNIQUE (email, phone_number)

💡 Best Practices
- Use composite keys only when natural uniqueness comes from multiple fields.
- Always document composite constraints for clarity.

🎯 Interview Question

Q: What’s the difference between composite primary key and separate unique constraints?

A: Composite PK combines columns as a single identifier, while unique constraints ensure combinations are distinct but don’t act as identifiers.

----------------



### 🔄 Bonus: Modify Constraints

In [ ]:
-- Add NOT NULL
ALTER TABLE users ALTER COLUMN email SET NOT NULL;

-- Drop CHECK
ALTER TABLE employees DROP CONSTRAINT employees_age_check;

-- Add Composite Unique
ALTER TABLE users ADD CONSTRAINT unique_email_phone UNIQUE (email, phone_number);


If you are confused about: `What is employees_age_check and where did it come from?`

In [ ]:
ALTER TABLE employees DROP CONSTRAINT employees_age_check;

✅ Explanation

When you create a CHECK constraint like this:

In [ ]:
CREATE TABLE employees (
    emp_id SERIAL PRIMARY KEY,
    age INT CHECK (age >= 18)
);

PostgreSQL automatically gives a default name to that CHECK constraint if you don't provide one explicitly.
That name is usually generated like:

So in our case:

- Table name: `employees`

- Column name: `age`

- Constraint type: `CHECK`

👉 PostgreSQL auto-generates the name:

That’s why, to remove that constraint, you write:

In [ ]:
ALTER TABLE employees DROP CONSTRAINT employees_age_check;

✅ Want to avoid default names?

You can give your own custom constraint name like this:

In [ ]:
CREATE TABLE employees (
    emp_id SERIAL PRIMARY KEY,
    age INT CONSTRAINT check_min_age CHECK (age >= 18)
);

Now if you want to drop it later, you’ll do:

In [ ]:
ALTER TABLE employees DROP CONSTRAINT check_min_age;